In [1]:
from data_loader import *
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

c:\Users\ASUS\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\ASUS\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [2]:
#we will load datasets first
data_loader = DataLoader("D:/Deloitte/DIH-X-AUC-Hackathon/data/Inventory Management")
additions_on_food = data_loader.load_csv('dim_add_ons.csv') #it contains the price and status and id and category and if it is deleted or not
campaigns_offers = data_loader.load_csv('dim_campaigns.csv') #it contains campaigns (offers) and their places and if it is active or not
stock_items_info = data_loader.load_csv('dim_items.csv')  #it contains items and if there is delivery or takeaway and if it is there and 
#types of additions and if it is displayed for customers
menu_additions = data_loader.load_csv('dim_menu_item_add_ons.csv') #specific add_ons to menu and the price and all are active
menu_items_info = data_loader.load_csv('dim_menu_items.csv') #it contains price and rating and votes and if it is acitve
places_info = data_loader.load_csv("dim_places.csv") #it contains  places and if it is bankrupt and visit_duration and vip threshold
#and waiting_time and binding_period and their title
stock_internal_measuring_units = data_loader.load_csv("dim_skus.csv") #units for measuring stock
stock_categories = data_loader.load_csv("dim_stock_categories.csv") #stock categories ids and titles
taxonomy_terms = data_loader.load_csv("dim_taxonomy_terms.csv") #contains ids that translates into vocab (male,female,bar,cafe,lounge)
users_info = data_loader.load_csv("dim_users.csv") #if account closed,name,country,currency,points,savings,and the role
bonus_codes = data_loader.load_csv("fct_bonus_codes.csv") #bonus ids, points, and duration
campaigns_info = data_loader.load_csv("fct_campaigns.csv") #campaigns titles and items, delivery, what original campaing,the place
#and redemptions and redemptions per_customer and if it is active or not.
#cash_balances = data_loader.load_csv("fct_cash_balances.csv")
#cash_balances.head() #it shows transactions and opening and closing balances
invoice_items = data_loader.load_csv("fct_invoice_items.csv") #it shows specific items with the amount (tablets)
order_items_info = data_loader.load_csv("fct_order_items.csv") #order_tiems and cost, commision, discount, points earned, redeemed,
#quanity, price and status and id of orders and title
orders_info  = data_loader.load_csv("fct_orders.csv") #orders cash amount, cashier, channel, points earned and redeemed, promise_time,
#source and status and total amount and type (takeaway or eat in)
most_ordered = data_loader.load_csv("most_ordered.csv") #most ordered items and count

Successfully loaded dim_add_ons.csv: 21101 rows
Successfully loaded dim_campaigns.csv: 641 rows
Successfully loaded dim_items.csv: 87266 rows
Successfully loaded dim_menu_item_add_ons.csv: 2459 rows
Successfully loaded dim_menu_items.csv: 30407 rows
Successfully loaded dim_places.csv: 1824 rows
Successfully loaded dim_skus.csv: 4 rows
Successfully loaded dim_stock_categories.csv: 3 rows
Successfully loaded dim_taxonomy_terms.csv: 904 rows
Successfully loaded dim_users.csv: 22762 rows
Successfully loaded fct_bonus_codes.csv: 6 rows
Successfully loaded fct_campaigns.csv: 641 rows
Successfully loaded fct_invoice_items.csv: 3124 rows
Successfully loaded fct_order_items.csv: 1974592 rows


d:\Deloitte\DIH-X-AUC-Hackathon\src\models\data_loader.py:56: DtypeWarning: Columns (10,17,18,26,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, parse_dates=parse_dates)


Successfully loaded fct_orders.csv: 399810 rows
Successfully loaded most_ordered.csv: 93048 rows


In [3]:
orders_info.drop(['cashier_notified','code','trainee_mode','type','demo_mode'],axis=1,inplace=True)
orders_info.isnull().sum()

id                                 0
user_id                            0
created                            0
updated                            0
updated_by                    398773
account_id                    397038
cash_amount                        0
channel                            0
customer_mobile_phone         398093
customer_name                 349974
delivery_charge                    1
delivery_location_id          399382
discount_amount                    1
driver_id                     399780
external_id                   398553
instructions                  398456
items_amount                       1
payment_method                     1
pickup_time                   399336
place_id                           1
points_earned                      1
points_redeemed                    1
promise_time                       1
rejection_reason              399716
referring_user_id             399810
service_charge                394512
source                             1
s

In [4]:
#merge orders and order_items to get customer metrics
customer_metrics = order_items_info.copy()
slice = orders_info[orders_info['id'].isin(customer_metrics['order_id'])]
slice = slice[['id', 'place_id', 'created', 'total_amount']]
slice.rename(columns={'id': 'order_id'}, inplace=True)
cols_to_drop = customer_metrics.columns.intersection(slice.columns).difference(['order_id'])
customer_metrics.drop(cols_to_drop,axis=1,inplace=True)
customer_metrics = customer_metrics.merge(
    slice, 
    on='order_id'
)
#merge customer metrics with places_info
slice= places_info[places_info['id'].isin(customer_metrics['place_id'])]
slice = slice[['id', 'waiting_time', 'vip_threshold']]
slice.rename(columns={'id': 'place_id'}, inplace=True)
cols_to_drop = customer_metrics.columns.intersection(slice.columns).difference(['place_id'])
customer_metrics.drop(cols_to_drop,axis=1,inplace=True)
customer_metrics = customer_metrics.merge(
    slice, 
    on='place_id'
)
#merge customer metrics with menu_items_info to get rating
slice= menu_items_info[menu_items_info['title'].isin(customer_metrics['title'])]
slice = slice[['title', 'rating']]
cols_to_drop = customer_metrics.columns.intersection(slice.columns).difference(['title'])
customer_metrics.drop(cols_to_drop,axis=1,inplace=True)
customer_metrics = customer_metrics.merge(
    slice, 
    on='title'
)
customer_metrics.head()

,Unnamed: 0,id,user_id,updated,title,campaign_id,commission_amount,cost,discount_amount,external_id,...,removed_ingredients,add_on_ids,status,vat_amount,place_id,created,total_amount,waiting_time,vip_threshold,rating
0,0,60824,512,2024-02-26,Latte,none,1.5,50.0,0.0,none,...,none,0,Pending,0.0,59897.0,1613139429,50.0,15,20,10
1,0,60824,512,2024-02-26,Latte,none,1.5,50.0,0.0,none,...,none,0,Pending,0.0,59897.0,1613139429,50.0,15,20,18
2,0,60824,512,2024-02-26,Latte,none,1.5,50.0,0.0,none,...,none,0,Pending,0.0,59897.0,1613139429,50.0,15,20,0
3,0,60824,512,2024-02-26,Latte,none,1.5,50.0,0.0,none,...,none,0,Pending,0.0,59897.0,1613139429,50.0,15,20,0
4,0,60824,512,2024-02-26,Latte,none,1.5,50.0,0.0,none,...,none,0,Pending,0.0,59897.0,1613139429,50.0,15,20,0


In [5]:
customer_metrics.columns.tolist()

['Unnamed: 0',
 'id',
 'user_id',
 'updated',
 'title',
 'campaign_id',
 'commission_amount',
 'cost',
 'discount_amount',
 'external_id',
 'group',
 'instructions',
 'item_id',
 'order_id',
 'points_earned',
 'points_redeemed',
 'price',
 'quantity',
 'redemptions',
 'removed_ingredients',
 'add_on_ids',
 'status',
 'vat_amount',
 'place_id',
 'created',
 'total_amount',
 'waiting_time',
 'vip_threshold',
 'rating']

In [6]:
customer_metrics = customer_metrics[['user_id','discount_amount','points_earned','points_redeemed','updated','price','quantity','waiting_time','vip_threshold','rating']]

In [7]:
selected_features = customer_metrics.groupby("user_id").agg({
    'discount_amount':"sum","points_earned":"sum","points_redeemed":"sum","price":"sum","waiting_time":"sum","vip_threshold":"first","rating":"mean"
})

In [11]:
#create the target
orders_dates = orders_info.copy()
orders_dates['order_date'] = pd.to_datetime(orders_dates['created'], errors='coerce')
recent_cutoff = orders_dates['order_date'].quantile(0.75)
early_cutoff = orders_dates['order_date'].quantile(0.25)
recent = orders_dates[orders_dates['order_date'] >= recent_cutoff].groupby('user_id').size()
early = orders_dates[orders_dates['order_date'] <= early_cutoff].groupby('user_id').size()
target = pd.DataFrame({'early_orders': early, 'recent_orders': recent}).fillna(0)
target['is_churned'] = (target['recent_orders'] < target['early_orders']).astype(int)


In [13]:
model_data = selected_features.join(target['is_churned'], how='inner').dropna()
model_data.to_csv("D:/Deloitte/DIH-X-AUC-Hackathon/ML_Models/customer_churn/finalized_data.csv")